**Imports:**

In [4]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

import seaborn as sns

In [115]:
def create_interactions_file(dataset="ml-latest-small", target=False):
  if not target: target = dataset
  os.makedirs(f"./{target}", exist_ok=True)
  
  interactions = pd.read_csv(f"./raw/{dataset}/ratings.csv", names=["user_id", "item_id", "","--"], usecols=[0,1], header=0)
  interactions["interaction"] = 1
  interactions.to_csv(f"./{target}/interaction_data.csv", index=False)
  return interactions

def create_neg_file(interactions, target):
  os.makedirs(f"./{target}", exist_ok=True)
  
  users = interactions['user_id'].unique()
  items = interactions['item_id'].unique()
  
  negative_samples = []
  for user in users:
    user_interacted_items = interactions[interactions['user_id'] == user]['item_id'].tolist()
    potential_neg_items = np.setdiff1d(items, user_interacted_items, assume_unique=True)
    
    negative_samples.append({'user_id': user, 'item_ids': potential_neg_items})
    
  negative_df = pd.DataFrame(negative_samples)
  negative_df.to_csv(f"./{target}/negative_samples.csv", index=False)
    
  return negative_df

def generate_negative_samples(user_id, negative, num_negatives=3):
    # Losowy wybór 10 unikalnych item_id
    negative_items = np.random.choice(np.concatenate(negative[negative["user_id"]==3]["item_ids"].values), size=num_negatives)
    
    # Tworzenie DataFrame z negatywnymi próbkami
    negative_samples = pd.DataFrame({
        'user_id': [user_id] * num_negatives,
        'item_id': negative_items,
        'interaction': [0] * num_negatives
    })
    
    return negative_samples
  
def create_train_test_files(interactions, negative, target, seed=43, train_negatives_num=1, test_negatives_num=100):
  os.makedirs(f"./{target}", exist_ok=True)
  
  unique_users = interactions['user_id'].unique()
  
  df = interactions.sample(frac=1).sort_values(by=['user_id']).reset_index(drop=True)

  test = df.groupby('user_id').tail(1).copy()
  train = df.drop(test.index).copy()

  train_negatives = pd.concat([generate_negative_samples(user_id, negative, train_negatives_num*len(df[df["user_id"]==user_id])) for user_id in unique_users])
  train = pd.concat([train, train_negatives]).sample(frac=1)
  
  test_negatives = pd.concat([generate_negative_samples(user_id, negative, test_negatives_num) for user_id in unique_users])
  test = pd.concat([test, test_negatives]).sample(frac=1)
  
  train.to_csv(f"./{target}/train_data_1-{train_negatives_num}.csv", index=False)
  test.to_csv(f"./{target}/test_data.csv", index=False)
  
  return train, test

In [ ]:
def create_items_ids_file(interactions, target):
    os.makedirs(f"./{target}", exist_ok=True)
    
    unique_items = interactions["item_id"].unique()
    items = pd.DataFrame(unique_items, columns=["item_id"])
    items["new_item_id"] = range(len(items))

    items.to_csv(f"./{target}/item_id_translation.csv", index=False)
    
    return items

def translate_item_id(interactions, item_id_translation, id_from, id_to, target):
    interactions_translated = interactions.merge(
        item_id_translation,
        left_on=id_from,
        right_on="item_id",
        how="left"
    )
    
    interactions_translated[id_from] = interactions_translated[id_to]
    
    interactions_translated = interactions_translated.drop(columns=["item_id"])
    interactions_translated = interactions_translated.rename(columns={"new_item_id": "item_id"})
    
    interactions_translated.to_csv(f"./{target}/interaction_data.csv", index=False)
    
    return interactions_translated
  
def create_user_ids_file(interactions, target):
    os.makedirs(f"./{target}", exist_ok=True)
    
    unique_items = interactions["user_id"].unique()
    items = pd.DataFrame(unique_items, columns=["user_id"])
    items["new_user_id"] = range(len(items))

    items.to_csv(f"./{target}/user_id_translation.csv", index=False)
    
    return items

def translate_user_ids(interactions, item_id_translation, id_from, id_to, target):
    interactions_translated = interactions.merge(
        item_id_translation,
        left_on=id_from,
        right_on="user_id",
        how="left"
    )
    
    interactions_translated[id_from] = interactions_translated[id_to]
    
    interactions_translated = interactions_translated.drop(columns=["user_id"])
    interactions_translated = interactions_translated.rename(columns={"new_user_id": "user_id"})
    
    interactions_translated.to_csv(f"./{target}/interaction_data.csv", index=False)
    
    return interactions_translated

In [133]:

  
interactions = create_interactions_file("ml-latest-small")
item_id_translation = create_items_ids_file(interactions, target="ml-latest-small")
interactions = translate_item_id(interactions, item_id_translation, "item_id", "new_item_id", target="ml-latest-small")

user_id_translation = create_user_ids_file(interactions, target="ml-latest-small")
interactions = translate_user_ids(interactions, user_id_translation, "user_id", "new_user_id", target="ml-latest-small")

negative = create_neg_file(interactions, "ml-latest-small")
train, test = create_train_test_files(interactions, negative, target="ml-latest-small", train_negatives_num=10, test_negatives_num=100)

print(f"train shape: {train.shape}")
display(train.describe())
print(f"test shape: {test.shape}")
display(test.describe())

train shape: (1108575, 3)


,interaction,item_id,user_id
count,1.108575e+06,1.108575e+06,1.108575e+06
mean,9.040886e-02,4.686049e+03,3.251421e+02
std,2.867668e-01,2.838464e+03,1.826186e+02
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,2.159000e+03,1.760000e+02
50%,0.000000e+00,4.607000e+03,3.240000e+02
75%,0.000000e+00,7.148000e+03,4.760000e+02
max,1.000000e+00,9.723000e+03,6.090000e+02


test shape: (61610, 3)


,interaction,item_id,user_id
count,61610.000000,61610.000000,61610.000000
mean,0.009901,4935.635270,304.500000
std,0.099011,2765.785566,176.093025
min,0.000000,0.000000,0.000000
25%,0.000000,2546.000000,152.000000
50%,0.000000,4937.000000,304.500000
75%,0.000000,7321.000000,457.000000
max,1.000000,9723.000000,609.000000
